## The first big project - Professionally You!

### And, Tool use.

### But first: introducing Pushover

Pushover is a nifty tool for sending Push Notifications to your phone.

It's super easy to set up and install!

Simply visit https://pushover.net/ and click 'Login or Signup' on the top right to sign up for a free account, and create your API keys.

Once you've signed up, on the home screen, click "Create an Application/API Token", and give it any name (like Agents) and click Create Application.

Then add 2 lines to your `.env` file:

PUSHOVER_USER=_put the key that's on the top right of your Pushover home screen and probably starts with a u_  
PUSHOVER_TOKEN=_put the key when you click into your new application called Agents (or whatever) and probably starts with an a_

Finally, click "Add Phone, Tablet or Desktop" to install on your phone.

## In this lab we will still implement the Knowing You solution we implemented in the last lab by passing knowledge base about the author in the form of the linked-in profile in pdf format and a summary description. But this time we will use tools.

**These tools will send a notification to our phone therefore we will use the pushover app .**

**The goal in this lab is to see how the tools that we pass are not something magical rather it is simply a simple workflow hidden under abstractions of the agent workflow**

**Again , tools are not being run by the LLMs , rather they are being run by us , LLMs only predict the next token**

In [5]:
# imports

from dotenv import load_dotenv
from openai import OpenAI
import json
import os
import requests
from pypdf import PdfReader
import gradio as gr

In [7]:
# The usual start

load_dotenv(override=True)
openai = OpenAI()

In [8]:
# For pushover

pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"

## This is a simple way to Push notification through pushover

In [9]:
def push(message):
    print(f"Push: {message}")
    payload = {"user": pushover_user, "token": pushover_token, "message": message}
    requests.post(pushover_url, data=payload)

In [10]:
push("HEY!!")

Push: HEY!!


## Below are two tools that we will equip our LLMs with. Thereby making it capable of calling these tools

1. The first tool is record_user_details which will record when someone wants to get in touch with us.
2. The second tool will record when it does not know answer to a question that the user asks.

In both the cases the messages will be pushed as push notification to the pushover app.

In [11]:
def record_user_details(email, name="Name not provided", notes="not provided"):
    push(f"Recording interest from {name} with email {email} and notes {notes}")
    return {"recorded": "ok"}

In [12]:
def record_unknown_question(question):
    push(f"Recording {question} asked that I couldn't answer")
    return {"recorded": "ok"}

#### Previously we discussed that tool use is simply json and if statements. We will see that in detail now.

Below are the jsons where we are defining our tools.

There are standard fields in the json like name of the tool , description - gives an explanation on why to use this tool , parameters. 

**Description is one of the most important fields since based on this the LLM will decide to use the tool or not**

In [13]:
record_user_details_json = {
    "name": "record_user_details",
    "description": "Use this tool to record that a user is interested in being in touch and provided an email address",
    "parameters": {
        "type": "object",
        "properties": {
            "email": {
                "type": "string",
                "description": "The email address of this user"
            },
            "name": {
                "type": "string",
                "description": "The user's name, if they provided it"
            }
            ,
            "notes": {
                "type": "string",
                "description": "Any additional information about the conversation that's worth recording to give context"
            }
        },
        "required": ["email"],
        "additionalProperties": False
    }
}

Here we define the json with parameters like name of the tool , the description , parameters like input paramters for the tools which is a function and then describe these parameters.

For this tool where we want to record the user who is trying to contact us we have made email as a mandatory field.

In [14]:
record_unknown_question_json = {
    "name": "record_unknown_question",
    "description": "Always use this tool to record any question that couldn't be answered as you didn't know the answer",
    "parameters": {
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "The question that couldn't be answered"
            },
        },
        "required": ["question"],
        "additionalProperties": False
    }
}

Here is the json for the second tool where we again define keys like name of the tool , description of the tool , the input parameters , required etc.

### Then we list down the tools in a list of dictionary with each tool being labelled with a type called function here.

**Note - Here we provided the json of the functions and not the function name**

In [15]:
tools = [{"type": "function", "function": record_user_details_json},
        {"type": "function", "function": record_unknown_question_json}]

In [16]:
tools

[{'type': 'function',
  'function': {'name': 'record_user_details',
   'description': 'Use this tool to record that a user is interested in being in touch and provided an email address',
   'parameters': {'type': 'object',
    'properties': {'email': {'type': 'string',
      'description': 'The email address of this user'},
     'name': {'type': 'string',
      'description': "The user's name, if they provided it"},
     'notes': {'type': 'string',
      'description': "Any additional information about the conversation that's worth recording to give context"}},
    'required': ['email'],
    'additionalProperties': False}}},
 {'type': 'function',
  'function': {'name': 'record_unknown_question',
   'description': "Always use this tool to record any question that couldn't be answered as you didn't know the answer",
   'parameters': {'type': 'object',
    'properties': {'question': {'type': 'string',
      'description': "The question that couldn't be answered"}},
    'required': ['quest

## Why Do we Put the Description of the tools in Json ?

This is because the LLMs have been trained on a lot of json data therefore they can be easily understood by the LLM.

### Next we need to write a function that will run the actual function when the LLM responds with the response that please run the tool.

**This is the if condition we talked about previously/**

#### This function takes a list of tool_calls then runs them and then adds the result of the tool call to the result which then will be sent to the llm.

**Note - tool_calls is the structured output that we got from the LLM in json format.**

First in the function we will loop through the tool_calls. Typically we will call only one tool at a time but the construct supports running multiple tools.

 

Whichever tool the LLM wants us to run , it will return that tool to us in this json format.

In [17]:
# This function can take a list of tool calls, and run them. This is the IF statement!!

def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        #Fetching the arguments from the tool call json , this will be passed to the function.
        arguments = json.loads(tool_call.function.arguments)
        print(f"Tool called: {tool_name}", flush=True)
        print(f"Arguments: {arguments}", flush=True)
        
        # THE BIG IF STATEMENT!!! This is the if statement we talked about previously.
        # This is the example of the if statement that is implemented by agent frameworks under the hood.
        # Since we do not know the exact number of parameters we pass varargs arguments.
        if tool_name == "record_user_details":
            result = record_user_details(**arguments)
        elif tool_name == "record_unknown_question":
            result = record_unknown_question(**arguments)

        # Finally we added the result of the tool call to the results list with role as tool.
        results.append({"role": "tool","content": json.dumps(result),"tool_call_id": tool_call.id})
    return results

#### Below code is a way to use reflection to call the function based on the name present in the tool_call json , instead of using if statements.

global() returns us a dictionary with functions with the global scope.
Here we are looking up for the global function with name record_unknown_function and passing the argument.

In [18]:
globals()["record_unknown_question"]("this is a really hard question")

Push: Recording this is a really hard question asked that I couldn't answer


{'recorded': 'ok'}

In [ ]:
# Using above logic , This is a more elegant way that avoids the IF statement.

def handle_tool_calls(tool_calls):
    print(f"Tool calls: {tool_calls}", flush=True)
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Tool called: {tool_name}", flush=True)
        print(f"Arguments : {arguments}", flush=True)
        # This is the way to get the function from the global scope.
        # But it is still a if statement.
        tool = globals().get(tool_name)
        
        result = tool(**arguments) if tool else {}
        results.append({"role": "tool","content": json.dumps(result),"tool_call_id": tool_call.id})
    return results

#### We will still be using the linkedin profile in pdf and the summary text like we did in the last lab

In [20]:
reader = PdfReader("me/linkedin.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

name = "Ed Donner"

#### The system prompt is still same as the last lab except for the last two lines where we are asking the llm to call tools if required.

```
If you don't know the answer to any question, use your record_unknown_question tool to record the question that you couldn't answer, even if it's about something trivial or unrelated to career. \
If the user is engaging in discussion, try to steer them towards getting in touch via email; ask for their email and record it using your record_user_details tool. "
```

In [23]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer to any question, use your record_unknown_question tool to record the question that you couldn't answer, even if it's about something trivial or unrelated to career. \
If the user is engaging in discussion, try to steer them towards getting in touch via email; ask for their email and record it using your record_user_details tool. "

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


#### The chat function as required by gradio

1. The message is still same wherein we are appending the new message to the history along with system prompt.

2. Then we run a loop until the variable done is set to true
    1. Within the Loop , openai is called.
            a. We pass the model , the messages and the **tools**
            b. **tools** is the json structure we created defining our tools.

    2. Then comes the response.
            a. Response can either be the final response
            b. Or it could be an indication that the tools needs to be called.
            c. Therefore we see a field in the response finish reason which tells us if the llm wants us to run a tool.

    3. If the reason is tool_calls ,
        1. We pluck out the tool
        2. Then we call the handle_tool_call method with the tool_calls json.
        3. This is where we make the actual function call.
        4. Then in the set of messages we put the original prompt by the user so original question and the result from the tool call.

3. The loop executes again , then again openai is called , again it is checked if the finish reason is tool call

4. This process continues until the finish reason is not tool_calls. 
5. If so the done variable is set as True and the response is returned.
    

In [26]:
import re


def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    print(messages)
    done = False
    while not done:

        # This is the call to the LLM - see that we pass in the tools json

        response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages, tools=tools)

        print("Response from LLM is : ")
        print(response.choices[0])
        finish_reason = response.choices[0].finish_reason
        
        # If the LLM wants to call a tool, we do that!
         
        if finish_reason=="tool_calls":
            message = response.choices[0].message
            tool_calls = message.tool_calls
            results = handle_tool_calls(tool_calls)
            messages.append(message)
            messages.extend(results)
            print("Messages after tool call is : ")
            print(messages)
        else:
            done = True
    return response.choices[0].message.content

In [27]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


[{'role': 'system', 'content': "You are acting as Ed Donner. You are answering questions on Ed Donner's website, particularly questions related to Ed Donner's career, background, skills and experience. Your responsibility is to represent Ed Donner for interactions on the website as faithfully as possible. You are given a summary of Ed Donner's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer to any question, use your record_unknown_question tool to record the question that you couldn't answer, even if it's about something trivial or unrelated to career. If the user is engaging in discussion, try to steer them towards getting in touch via email; ask for their email and record it using your record_user_details tool. \n\n## Summary:\nMy name is Ed Donner. I'm an entrepreneur, software engineer and data scientist. I'm originall

#### Lets see the flow for below conversation -
User sends the message - Hi , Who is your favourite musician ?

**Since this data is not present in the knowledgebase that is linkedin profile and summary therefore the tools will be called.**

Lets see the flow -
message = Hi , Who is your favourite musician ?

1. chat() is invoked , the message gets added to the list messages.
        [{'role': 'system', 'content': "....}, {'role': 'user', 'content': 'Hi , Who is your favourite musician ?'}]


2. The while loop starts , the call to openai is made.
3. The llm responds with the finish_reason as tool_call
        Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_OvRtq1hHqZ6aou0lAqTdSm4Q', function=Function(arguments='{"question":"Who is Ed Donner\'s favourite musician?"}', name='record_unknown_question'), type='function')]))


4. The tool_call is plucked out from themessage and sent to the handle_tool_call()
Tool called: record_unknown_question

5. The method is invoked , then the notification is pushed.
Push: Recording Who is Ed Donner's favourite musician? asked that I couldn't answer

6. The response is appended to the message object
[
{'role': 'system', 'content': "..."}, 
{'role': 'user', 'content': 'Hi , Who is your favourite musician ?'}, 
ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_OvRtq1hHqZ6aou0lAqTdSm4Q', function=Function(arguments='{"question":"Who is Ed Donner\'s favourite musician?"}', name='record_unknown_question'), type='function')]), 

{'role': 'tool', 'content': '{"recorded": "ok"}', 'tool_call_id': 'call_OvRtq1hHqZ6aou0lAqTdSm4Q'}]

As we can see here the message returned from the handle_tool_call function is appended with the role as tool.

7. Again the for loop executes , this time the finish_reason is not tool_calll , therefore the output is returned to the user.


image.png

## And now for deployment

This code is in `app.py`

All of the above code that we tried out in this notebook is moved to the app.py file.
We define a class called Me which loads the knowledge bases. Then we also have the handle_tool_call function in this class.
Then we also have the chat() function required for gradio in this class as well as the gradio.interface function in the main function.

To run the app.py class in local we cal do `uv run app.py`

## Deploying to HuggingFace

We will deploy to HuggingFace Spaces. Thank you student Robert M for improving these instructions.

Before you start: remember to update the files in the "me" directory - your LinkedIn profile and summary.txt - so that it talks about you!  
Also check that there's no README file within the 1_foundations directory. If there is one, please delete it. The deploy process creates a new README file in this directory for you.

1. Visit https://huggingface.co and set up an account  
2. From the Avatar menu on the top right, choose Access Tokens. Choose "Create New Token". Give it WRITE permissions.
3. Take this token and add it to your .env file: `HF_TOKEN=hf_xxx` and see note below if this token doesn't seem to get picked up during deployment  
4. From the 1_foundations folder, enter: `uv run gradio deploy` and if for some reason this still wants you to enter your HF token, then interrupt it with ctrl+c and run this instead: `uv run dotenv -f ../.env run -- uv run gradio deploy` which forces your keys to all be set as environment variables   
5. Follow its instructions: name it "career_conversation", specify app.py, choose cpu-basic as the hardware, say Yes to needing to supply secrets, provide your openai api key, your pushover user and token, and say "no" to github actions.  

#### Extra note about the HuggingFace token

A couple of students have mentioned the HuggingFace doesn't detect their token, even though it's in the .env file. Here are things to try:   
1. Restart Cursor   
2. Rerun load_dotenv(override=True) and use a new terminal (the + button on the top right of the Terminal)   
3. In the Terminal, run this before the gradio deploy: `$env:HF_TOKEN = "hf_XXXX"`  
Thank you James and Martins for these tips.  

#### More about these secrets:

If you're confused by what's going on with these secrets: it just wants you to enter the key name and value for each of your secrets -- so you would enter:  
`OPENAI_API_KEY`  
Followed by:  
`sk-proj-...`  

And if you don't want to set secrets this way, or something goes wrong with it, it's no problem - you can change your secrets later:  
1. Log in to HuggingFace website  
2. Go to your profile screen via the Avatar menu on the top right  
3. Select the Space you deployed  
4. Click on the Settings wheel on the top right  
5. You can scroll down to change your secrets, delete the space, etc.

#### And now you should be deployed!

Here is mine: https://huggingface.co/spaces/ed-donner/Career_Conversation

I just got a push notification that a student asked me how they can become President of their country 😂😂

For more information on deployment:

https://www.gradio.app/guides/sharing-your-app#hosting-on-hf-spaces

To delete your Space in the future:  
1. Log in to HuggingFace
2. From the Avatar menu, select your profile
3. Click on the Space itself and select the settings wheel on the top right
4. Scroll to the Delete section at the bottom
5. ALSO: delete the README file that Gradio may have created inside this 1_foundations folder (otherwise it won't ask you the questions the next time you do a gradio deploy)


<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Exercise</h2>
            <span style="color:#ff7800;">• First and foremost, deploy this for yourself! It's a real, valuable tool - the future resume..<br/>
            • Next, improve the resources - add better context about yourself. If you know RAG, then add a knowledge base about you.<br/>
            • Add in more tools! You could have a SQL database with common Q&A that the LLM could read and write from?<br/>
            • Bring in the Evaluator from the last lab, and add other Agentic patterns.
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Commercial implications</h2>
            <span style="color:#00bfff;">Aside from the obvious (your career alter-ego) this has business applications in any situation where you need an AI assistant with domain expertise and an ability to interact with the real world.
            </span>
        </td>
    </tr>
</table>